In [1]:
import requests

# Includes API Key and username for authentication purposes
API_KEY = '8cd2572cd7ed88b8fa239ea6d041f9ea'
USER_AGENT = 'thisisbigcat'

headers = {
    'user_agent': USER_AGENT
}

# Includes the parameters necessary to access the JSON
payload = {
    'api_key': API_KEY,
    'method': 'user.getfriends',
    'user': 'Erkan-Yilmaz',
    'format': 'json'
}

# Makes a request to the API root with the necessary parameters (status code to confirm working access)
r = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=payload)
r.status_code

200

In [2]:
# Function to call different methods more easily
def lastfm_get(payload):
    # define headers and URL
    headers = {'user-agent': USER_AGENT}
    url = 'https://ws.audioscrobbler.com/2.0/'

    # Add API key and format to the payload
    payload['api_key'] = API_KEY
    payload['format'] = 'json'

    response = requests.get(url, headers=headers, params=payload)
    return response

In [3]:
# Testing function for proper response
r = lastfm_get({
    'method': 'user.getfriends',
    'user': 'Erkan-Yilmaz'
})

r.status_code

200

In [5]:
import json

def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

jprint(r.json())

{
    "friends": {
        "@attr": {
            "page": "1",
            "perPage": "50",
            "total": "7940",
            "totalPages": "159",
            "user": "Erkan-Yilmaz"
        },
        "user": [
            {
                "bootstrap": "0",
                "country": "Australia",
                "image": [
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/34s/920d0d72df217011d8fdf9da82cb3a38.png",
                        "size": "small"
                    },
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/64s/920d0d72df217011d8fdf9da82cb3a38.png",
                        "size": "medium"
                    },
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/174s/920d0d72df217011d8fdf9da82cb3a38.png",
                        "size": "large"
                    },
                    {
                        "#text": "https://l

In [6]:
# Prints out the attributes of the friends list
jprint(r.json()['friends']['@attr'])

{
    "page": "1",
    "perPage": "50",
    "total": "7940",
    "totalPages": "159",
    "user": "Erkan-Yilmaz"
}


In [7]:
# Calls in the requests_cache for improved perfomance on requests functions
import requests_cache

requests_cache.install_cache()

In [8]:
import time
from IPython.core.display import clear_output

responses = []

page = 1
total_pages = 99999 # this is just a dummy number so the loop starts

while page <= total_pages:
    payload = {
        'method': 'user.getfriends',
        'limit': 500,
        'user': 'Erkan-Yilmaz', 
        'page': page
    }

    # print some output so we can see the status
    print("Requesting page {}/{}".format(page, total_pages))
    # clear the output to make things neater
    clear_output(wait = True)

    # make the API call
    response = lastfm_get(payload)

    # if we get an error, print the response and halt the loop
    if response.status_code != 200:
        print(response.text)
        break

    # extract pagination info
    page = int(response.json()['friends']['@attr']['page'])
    total_pages = int(response.json()['friends']['@attr']['totalPages'])

    # append response
    responses.append(response)

    # if it's not a cached result, sleep
    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)

    # increment the page number
    page += 1

Requesting page 16/16


In [9]:
import pandas as pd

# Processes the data
r0 = responses[0]
r0_json = r0.json()
r0_friends = r0_json['friends']['user']
r0_df = pd.DataFrame(r0_friends)
r0_df.head()

,name,url,country,playlists,playcount,image,registered,realname,subscriber,bootstrap,type
0,Oliwally2,https://www.last.fm/user/Oliwally2,Australia,0,0,"[{'size': 'small', '#text': 'https://lastfm.fr...","{'unixtime': '1482204132', '#text': '2016-12-2...",Oliver,0,0,user
1,JonObi,https://www.last.fm/user/JonObi,United Kingdom,0,0,"[{'size': 'small', '#text': 'https://lastfm.fr...","{'unixtime': '1215423754', '#text': '2008-07-0...",Jon Mills,0,0,user
2,SfromL,https://www.last.fm/user/SfromL,United Kingdom,0,0,"[{'size': 'small', '#text': 'https://lastfm.fr...","{'unixtime': '1148722906', '#text': '2006-05-2...",Sean Hornby,1,0,subscriber
3,bloody72,https://www.last.fm/user/bloody72,Poland,0,0,"[{'size': 'small', '#text': 'https://lastfm.fr...","{'unixtime': '1214431908', '#text': '2008-06-2...",krzysztof pluta,0,0,user
4,individuaali,https://www.last.fm/user/individuaali,Finland,0,0,"[{'size': 'small', '#text': 'https://lastfm.fr...","{'unixtime': '1289418534', '#text': '2010-11-1...",,0,0,user


In [11]:
# Ensures the data is collected as one dataframe
frames = [pd.DataFrame(r.json()['friends']['user']) for r in responses]
friends = pd.concat(frames)
friends.head()

,name,url,country,playlists,playcount,image,registered,realname,subscriber,bootstrap,type
0,Oliwally2,https://www.last.fm/user/Oliwally2,Australia,0,0,"[{'size': 'small', '#text': 'https://lastfm.fr...","{'unixtime': '1482204132', '#text': '2016-12-2...",Oliver,0,0,user
1,JonObi,https://www.last.fm/user/JonObi,United Kingdom,0,0,"[{'size': 'small', '#text': 'https://lastfm.fr...","{'unixtime': '1215423754', '#text': '2008-07-0...",Jon Mills,0,0,user
2,SfromL,https://www.last.fm/user/SfromL,United Kingdom,0,0,"[{'size': 'small', '#text': 'https://lastfm.fr...","{'unixtime': '1148722906', '#text': '2006-05-2...",Sean Hornby,1,0,subscriber
3,bloody72,https://www.last.fm/user/bloody72,Poland,0,0,"[{'size': 'small', '#text': 'https://lastfm.fr...","{'unixtime': '1214431908', '#text': '2008-06-2...",krzysztof pluta,0,0,user
4,individuaali,https://www.last.fm/user/individuaali,Finland,0,0,"[{'size': 'small', '#text': 'https://lastfm.fr...","{'unixtime': '1289418534', '#text': '2010-11-1...",,0,0,user


In [12]:
# Modifies the datafram to only include usernames
friends = friends[['name']]
friends.head()

,name
0,Oliwally2
1,JonObi
2,SfromL
3,bloody72
4,individuaali


In [13]:
# Displays the info from the friends sample size
friends.info()
friends.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7939 entries, 0 to 439
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    7939 non-null   object
dtypes: object(1)
memory usage: 124.0+ KB


,name
count,7939
unique,7939
top,Oliwally2
freq,1


In [14]:
# Ensures that duplicate usernames are dropped
friends = friends.drop_duplicates().reset_index(drop=True)
friends.describe()

,name
count,7939
unique,7939
top,Oliwally2
freq,1


In [15]:
# Converts the usernames column into a list to be used
username_list = friends['name'].values.tolist()
username_list

['Oliwally2',
 'JonObi',
 'SfromL',
 'bloody72',
 'individuaali',
 'jamessurette',
 'MiTunesRock',
 'allexhjp',
 'karlososhd',
 'KevinReis',
 'mynameisjoannes',
 'Cydia9000',
 'luisAlberth',
 'guidedbyalan',
 'csullivan002',
 'Saddam_',
 'Yung_cum',
 'McYaballow',
 'FontaineLeglou',
 'Chazzy-Thrash',
 'xxx_666_',
 'red-corner',
 'Nnnas',
 'ThunderShock',
 'slayerdavid9',
 'thagreatmagnet',
 'deology',
 'HShhhh',
 'popzer',
 'yanwen',
 'Mahnam-E',
 'Neonkaa',
 'jess420ica',
 'InnocuousJam65',
 'markhertog1',
 'MelineCrolla',
 'guiusername',
 'MakkeBa',
 'ernstvdleeden',
 'Sad-Pacino',
 'maryam_feiz',
 'michciko',
 'Iggy_86',
 'Tarkovsky86',
 'RxLitschauer',
 'Fiberz',
 'LindsayVL',
 'dalegion',
 'zxcvbnm81',
 'kmi25',
 'Terence5555',
 'dharmeshwar',
 'Sepulchral_',
 'Belll333',
 'ezears',
 'AlexanderPope',
 'lu_rodrigo',
 'Meli_Smith',
 'poohlee',
 'fakoflan',
 'ojuaara',
 'mikedd',
 'Metalfight667',
 'douglaasp',
 'Bianca_Santiago',
 'Lucas_Ax',
 'MatValent11',
 'baartec',
 'onlycharle

In [16]:
# Function to lookup a user's loved tracks
def lookup_lovedtracks(user):
    response = lastfm_get({
        'method': 'user.getlovedtracks',
        'user':  user
    })

    # if there's an error, just return nothing
    if response.status_code != 200:
        return None

    # extract the top three tags and turn them into a string
    lovedtracks = [t['name'] for t in response.json()['lovedtracks']['track']]
    lovedtracks_str = ', '.join(lovedtracks)

    # rate limiting
    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)
    return lovedtracks_str

In [17]:
# Testing the new function
lookup_lovedtracks('rj')

"Fire, Just Jammin', Giddy Up, The Nod, Lady Day and John Coltrane, Ain't Got Time To Waste, Hold This Down (feat. Dean Curtis), Dancing on My Own, The Boston Rag, You Need Love, Don't You Know, Hey Laura, Mexico, Pretty Wasted, Axel F, The Future Soon, Jack Sparrow, I'm On A Boat, Dick In A Box, Jungle - Feat. Maverick Sabre, Baby Got Back, Stroker Ace, East Side Soldier (feat. Yukmouth, Dru Down & Pooh Man), Ain't No Love, Hypnotize, Aint Coming Home Tonight, Tequila Makes Her Clothes Fall Off, Pickup Man, Beer or Gasoline, Rain Is a Good Thing, Somethin' 'Bout a Truck, All Over the Road, Almost Like the Blues, Northern Lights, Hit The Road Jack, That Lady - Pts. 1 & 2, Wandering Eye - Radio Edit, Silver Rain (feat. Joe Kibble), Silver Rain, The Boogie Back, Boogie Stupid, Just Don't Want To Be Lonely, Funk in the Hole, What A Job, Cissy Strut, Romantic, The red one, Doctor Doctor, I Got The..., I Got The... - 2006 Digital Remaster"

In [18]:
# Collects the loved tracks data from the user
from tqdm import tqdm

responses = []

for i in tqdm(range(len(username_list))):
    responses.append(lookup_lovedtracks(username_list[i])) 

100%|██████████| 7939/7939 [00:41<00:00, 192.81it/s]


In [19]:
# Printing out the results
responses

["VPN, LONG SEASON - Live, Styrofoam, twin tip, PARAGRAPH OF MY LIFE, Oedipus Rex, Old Person Controller, Lookin 4 tha Suckin, Last Breath (Outroduction), Rather Be Broke, Columns, Collapse And Appear, 6 Foot 7 Foot, Youth Problem, That fuzz pedal you planted in your throat, its screw has started to come loose Your next effects pedal is up to you do you have it ready?, Huit octobre 1971, Schizophrenia, Kommienezuspadt, Walls of Sound, Fuckin Wit Dis Click, Paid in Full, You'd Be So Nice To Come Home To, African Violets, Mind Playing Tricks on Me, Ms. Jackson, Roses, So Fresh, So Clean, Hey Ya!, Jenifa Taught Me (Derwin's Revenge), Anna Got Me Clickin', Smoking on a J, Rag Mala In Jogia, Tango Till They're Sore, Misery Is the River of the World, Filth and Satan, Tetras, Table Top Joe, Theraps, Fawn, Sanity, For Adolphe Sax, Morning Glory, ILoveUIHateU, Shoota (feat. Lil Uzi Vert), My Favorite Ladies, All Outta Ale, Break You Off (feat. Reddog), All My Life, Rock The Clock, Science Ficti

In [20]:
# Organized into a dataframe
col_names = ['lovedtracks']
df = pd.DataFrame(responses, columns=col_names)
df

,lovedtracks
0,"VPN, LONG SEASON - Live, Styrofoam, twin tip, ..."
1,"Jean Vigo, what does it all mean?, Blowing It,..."
2,"Modern Parlance, Bomb Disneyland - Live At The..."
3,"Divine Hammer, Fruition, Hallowed Be Thy Name,..."
4,
...,...
7934,"PARTY, Little ms sunshine, Mona Lisa (Demo On ..."
7935,"Shinawi, Kwansan Yungma (Sôdo Sori folk song),..."
7936,"My Elastic Eye, Bullet With Butterfly Wings (O..."
7937,"I Care 4 U, Nasty, Breathe Again, Runaway, I'm..."


In [21]:
# Usernames inserted into the dataframe
df.insert(0, 'username', username_list)
df

,username,lovedtracks
0,Oliwally2,"VPN, LONG SEASON - Live, Styrofoam, twin tip, ..."
1,JonObi,"Jean Vigo, what does it all mean?, Blowing It,..."
2,SfromL,"Modern Parlance, Bomb Disneyland - Live At The..."
3,bloody72,"Divine Hammer, Fruition, Hallowed Be Thy Name,..."
4,individuaali,
...,...,...
7934,floriankuchan,"PARTY, Little ms sunshine, Mona Lisa (Demo On ..."
7935,luvandfeelins,"Shinawi, Kwansan Yungma (Sôdo Sori folk song),..."
7936,sezaun,"My Elastic Eye, Bullet With Butterfly Wings (O..."
7937,nils1983,"I Care 4 U, Nasty, Breathe Again, Runaway, I'm..."


In [41]:
# Expands the loved tracks into 50 columns 
expanded_df = df['lovedtracks'].str.split(', ', expand=True)
expanded_df.columns = [f'lovedtrack_{i+1}' for i in range(expanded_df.shape[1])]
df_expanded = pd.concat([df, expanded_df], axis=1)
df_expanded

,username,lovedtracks,lovedtrack_1,lovedtrack_2,lovedtrack_3,lovedtrack_4,lovedtrack_5,lovedtrack_6,lovedtrack_7,lovedtrack_8,...,lovedtrack_106,lovedtrack_107,lovedtrack_108,lovedtrack_109,lovedtrack_110,lovedtrack_111,lovedtrack_112,lovedtrack_113,lovedtrack_114,lovedtrack_115
0,Oliwally2,"VPN, LONG SEASON - Live, Styrofoam, twin tip, ...",VPN,LONG SEASON - Live,Styrofoam,twin tip,PARAGRAPH OF MY LIFE,Oedipus Rex,Old Person Controller,Lookin 4 tha Suckin,...,None,None,None,None,None,None,None,None,None,None
1,JonObi,"Jean Vigo, what does it all mean?, Blowing It,...",Jean Vigo,what does it all mean?,Blowing It,All of the Lights,Recollections of the Wraith,Swerve... the Reeping of All That Is Worthwhil...,All Tan Everything (feat. Jay-Z) (Produced by ...,Do You Realize??,...,None,None,None,None,None,None,None,None,None,None
2,SfromL,"Modern Parlance, Bomb Disneyland - Live At The...",Modern Parlance,Bomb Disneyland - Live At The Shacklewell,Promo No. 2 (Mind Review '89) - feat. Q-Tip,GMF,Come on Waleed (feat. Henry Rollins & Liz Stri...,Memory of a Free Festival - 2015 Remaster,Novocaine for the Soul,Resurrection,...,None,None,None,None,None,None,None,None,None,None
3,bloody72,"Divine Hammer, Fruition, Hallowed Be Thy Name,...",Divine Hammer,Fruition,Hallowed Be Thy Name,Sorry You're Not a Winner,Lip Gloss And Black,Caroline,Running In Menace - 2023 Version,Love Doesn't Live Here Anymore,...,None,None,None,None,None,None,None,None,None,None
4,individuaali,,,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7934,floriankuchan,"PARTY, Little ms sunshine, Mona Lisa (Demo On ...",PARTY,Little ms sunshine,Mona Lisa (Demo On Radio),State of Grace,Dramatic,Change Your Mind (No Seas Cortes),Coupure Électrique,Baby you..,...,None,None,None,None,None,None,None,None,None,None
7935,luvandfeelins,"Shinawi, Kwansan Yungma (Sôdo Sori folk song),...",Shinawi,Kwansan Yungma (Sôdo Sori folk song),Ajaeng sanjo,'O Mare E Tu,Bel El Madhi,Hayati,Dreamy Garden (Dus Bahcesi),Menari Hwansanggok (A Fantasy for the Menari T...,...,None,None,None,None,None,None,None,None,None,None
7936,sezaun,"My Elastic Eye, Bullet With Butterfly Wings (O...",My Elastic Eye,Bullet With Butterfly Wings (Original Single E...,Shut 'em Up (feat. Public Enemy & Manfred Mann),Petze,Papierkrieg,Gerechtigkeit,Der Partisan,Hey,...,None,None,None,None,None,None,None,None,None,None
7937,nils1983,"I Care 4 U, Nasty, Breathe Again, Runaway, I'm...",I Care 4 U,Nasty,Breathe Again,Runaway,I'm On My Way,In the Middle,She Wolf,Please Don't Leave Me,...,None,None,None,None,None,None,None,None,None,None


In [53]:
# Reduced the datafram down to 50 loved tracks columns and drops the 'lovedtracks' column
df_reduced = df_expanded.iloc[:, :52]
df_final = df_reduced.drop('lovedtracks', axis=1)
df_final

,username,lovedtrack_1,lovedtrack_2,lovedtrack_3,lovedtrack_4,lovedtrack_5,lovedtrack_6,lovedtrack_7,lovedtrack_8,lovedtrack_9,...,lovedtrack_41,lovedtrack_42,lovedtrack_43,lovedtrack_44,lovedtrack_45,lovedtrack_46,lovedtrack_47,lovedtrack_48,lovedtrack_49,lovedtrack_50
0,Oliwally2,VPN,LONG SEASON - Live,Styrofoam,twin tip,PARAGRAPH OF MY LIFE,Oedipus Rex,Old Person Controller,Lookin 4 tha Suckin,Last Breath (Outroduction),...,Fawn,Sanity,For Adolphe Sax,Morning Glory,ILoveUIHateU,Shoota (feat. Lil Uzi Vert),My Favorite Ladies,All Outta Ale,Break You Off (feat. Reddog),All My Life
1,JonObi,Jean Vigo,what does it all mean?,Blowing It,All of the Lights,Recollections of the Wraith,Swerve... the Reeping of All That Is Worthwhil...,All Tan Everything (feat. Jay-Z) (Produced by ...,Do You Realize??,Convexations,...,Rising Up (feat. Wale & Chrisette Michele),99 Problems,Sabotage,Root Down,Doin' It Again,Right On (feat. Joanna Newsom & STS),Shot Down,Pessimist,Hidden Wheel,Trip II The Moon (Kaleidoscopiklimax)
2,SfromL,Modern Parlance,Bomb Disneyland - Live At The Shacklewell,Promo No. 2 (Mind Review '89) - feat. Q-Tip,GMF,Come on Waleed (feat. Henry Rollins & Liz Stri...,Memory of a Free Festival - 2015 Remaster,Novocaine for the Soul,Resurrection,The Wrong Way To Be,...,Luvin Time,L.A. Woman,Vulture,Get What You Deserve,Radio Sit-In,Crooked Head,When You Sing,What You Need Is Jesus,You're to Good for Me,Elephant Man
3,bloody72,Divine Hammer,Fruition,Hallowed Be Thy Name,Sorry You're Not a Winner,Lip Gloss And Black,Caroline,Running In Menace - 2023 Version,Love Doesn't Live Here Anymore,Verstrahlt,...,1-800-CLOSE-UR-EYES,Initiation,Intrusive,Coming Storm,Puritania,Crawling Through Knives,Shining Of Your Soul,Advent,GNOSTIC FLESH/MORTAL HELL,Section
4,individuaali,,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7934,floriankuchan,PARTY,Little ms sunshine,Mona Lisa (Demo On Radio),State of Grace,Dramatic,Change Your Mind (No Seas Cortes),Coupure Électrique,Baby you..,FUTURE LOVER-未来恋人-,...,Roar,Movin' on without you,GALAXY SUPERNOVA,Realize,Weekend High,TOUCH DOWN,BAD,Radioactive,Alive,Liebe ist meine Rebellion
7935,luvandfeelins,Shinawi,Kwansan Yungma (Sôdo Sori folk song),Ajaeng sanjo,'O Mare E Tu,Bel El Madhi,Hayati,Dreamy Garden (Dus Bahcesi),Menari Hwansanggok (A Fantasy for the Menari T...,Sugung Ga,...,None,None,None,None,None,None,None,None,None,None
7936,sezaun,My Elastic Eye,Bullet With Butterfly Wings (Original Single E...,Shut 'em Up (feat. Public Enemy & Manfred Mann),Petze,Papierkrieg,Gerechtigkeit,Der Partisan,Hey,das ist nicht der Tag zu geh'n,...,Underclass Hero,8 200 200,La Ferme,Les Bottes Rouges,Happy Violentine,Cloud,Shooting From The Hip,The World of Jenny Brown,Chinese Takeaway,Le Roi Des Fourmis
7937,nils1983,I Care 4 U,Nasty,Breathe Again,Runaway,I'm On My Way,In the Middle,She Wolf,Please Don't Leave Me,Stars Are Blind,...,None,None,None,None,None,None,None,None,None,None


In [54]:
# Converts the df into a csv file
df_final.to_csv('lovedtracks.csv', index=False)